# Visualizing and Analyzing Meshes


When trying to understand the fine 3d morphology of a neuron (e.g. features under 1 micron in scale), meshes are a particularly useful representation. More precisely, a mesh is a collection of vertices and faces that define a 3d surface. The exact meshes that one sees in Neuroglancer can also be loaded for analysis and visualization in other tools.

# Downloading Meshes

The easiest tool for downloading MICrONs meshes is [Meshparty](https://github.com/sdorkenw/MeshParty), which is a python module that can be installed with `pip install meshparty`. Documentation for Meshparty can be found here.

Once installed, the typical way of getting meshes is by using a “MeshMeta” client that is told both the internet location of the meshes (`cv_path`) and a local directory in which to store meshes (`disk_cache_path`). Once initialized, the MeshMeta client can be used to download meshes for a given segmentation using its root id (`seg_id`). The following code snippet shows how to download an example mesh using a directory “meshes” as the local storage folder.

### Why MeshMeta?

One convenience of using the `MeshMeta` approach is that if you have already downloaded a mesh for with a given root id, it will be loaded from disk rather than re-downloaded.

If you have to download many meshes, it is somewhat faster to use the bulk `download_meshes` function and use multiple threads via the `n_threads` argument. If you download them to the same folder used for the MeshMeta object, they can be loaded through the same interface.

#### Note: Meshes can be hundreds of megabytes in size, so be careful about downloading too many if the internet is not acting well or your computer doesn’t have much disk space!

# Properties

Meshes have a large number of properties, many of which come from being based on the [Trimesh](https://trimsh.org/) library’s mesh format, and others being specific to MeshParty.

Several of the most important properties are:

- `mesh.vertices` : An `N x 3` list of vertices and their 3d location in nanometers, where N is the number of vertices.

- `mesh.faces` : An `P x 3` list of integers, with each row specifying a triangle of connected vertex indices.

- `mesh.edges` : An `M x 2` list of integers, with each row specifying a pair of connected vertex indices based off of faces.

- `mesh.link_edges` : An `M_l x 2` list of integers, with each row specifying a pair of “link edges” that were used to heal gaps based on proofreading edits.

- `mesh.graph_edges` : An `(M+M_l) x 2` list of integers, with each row specifying a pair of graph edges, which is the collection of both mesh.edges and `mesh.link_edges`.

- `mesh.csgraph` : A [Scipy Compressed Sparse Graph](https://docs.scipy.org/doc/scipy/reference/sparse.csgraph.html) representation of the mesh as an `NxN` graph of vertices connected to one another using graph edges and with edge weights being the distance between vertices. This is particularly useful for computing shortest paths between vertices.


#### Important: 
MICrONs meshes are not generally “watertight”, a property that would enable a number of properties to be computed natively by Trimesh. Because of this, Trimesh-computed properties relating to solid forms or volumes like `mesh.volume` or `mesh.center_mass` do not have sensible values and other approaches should be taken. Unfortunately, because of the Trimesh implementation of these properties it is up to the user to be aware of this issue.

# Visualizing

There are a variety of tools for visualizing meshes in python. MeshParty interfaces with VTK, a powerful but complex data visualization library that does not always work well in python. The basic pattern for MeshParty’s VTK integration is to create one or more “actors” from the data, and then pass those to a renderer that can be displayed in an interactive approach. The following code snippet shows how to visualize a mesh using this approach.

#### Note: By default, neurons will appear upside down because the coordinate system of the dataset has the y-axis value increasing along the “downward” pia to white matter axis. More documentation on the MeshParty VTK visualization can be [found here](https://meshparty.readthedocs.io/en/latest/source/meshparty.html).

Other tools worth exploring are [PyVista](https://docs.pyvista.org/), [Polyscope](https://polyscope.run/), [Vedo](https://vedo.embl.es/), [MeshLab](https://www.meshlab.net/), and if you have some existing experience, [Blender](https://www.blender.org/) (see Blender integration by our friends behind [Navis](https://navis.readthedocs.io/en/latest/source/blender.html), a fly-focused framework analyzing connectomics data).

# Masking

One of the most common operations on meshes is to mask them to a particular region of interest. This can be done by “masking” the mesh with a boolean array of length `N` where `N` is the number of vertices in the mesh, with `True` where the vertex should be kept and `False` where it should be omitted. There are several convenience functions to generate common masks in the [Mesh Filters](https://meshparty.readthedocs.io/en/latest/source/meshparty.html#module-meshparty.mesh_filters) module.

In the following example, we will first mask out all vertices that aren’t part of the largest connected component of the mesh (i.e. get rid of floating vertices that might arise due to internal surfaces) and then mask out all vertices that are more than 15,000 nm away from the soma center.

# Task 1

- Start with one neuron with a root id [from this google sheet](https://docs.google.com/spreadsheets/d/1adZ6f8zx691_mgQxFlTHUI0mhlNiIDtyobdt6CSVNBM/edit?usp=sharing).
- Download the mesh using meshparty
- Using trimesh_vtk, create a mesh actor and render your mesh of interest. What do you notice? Are there any artifacts that look suspicious? Would you feel confident extracting shape, size, and other morphological features from this object?
- Using mesh_filters, filter your mesh for the largest component and visualize it using vtk and compare the meshes before and after filtering.

#### Tip 1: To double check you have the right neuron, use your experience from session 1 to visualize your neuron in Neuroglancer.

#### Tip 2: When inspecting your mesh, it can be helpful to lower the opacity of your mesh to see any internal surfaces present

In [1]:
import os
from meshparty import trimesh_io, mesh_filters, trimesh_vtk
from caveclient import CAVEclient

/Users/leilae/anaconda3/envs/micronsbinder/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/Users/leilae/anaconda3/envs/micronsbinder/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/Users/leilae/anaconda3/envs/micronsbinder/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/Users/leilae/anaconda3/envs/micronsbinder/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [2]:
client = CAVEclient('minnie65_public')

In [3]:
mm = trimesh_io.MeshMeta(
    cv_path = client.info.segmentation_source(),
    disk_cache_path = 'meshes'
)

In [4]:
neuron_id = 864691135698459925

In [5]:
mesh = mm.mesh(seg_id=neuron_id)
mesh.vertices.shape, mesh.edges.shape

((1573926, 3), (9413070, 2))

In [8]:
neuron_actor = trimesh_vtk.mesh_actor(mesh,
                                     opacity = 0.8,
                                     color = (0,1,1))

trimesh_vtk.render_actors([neuron_actor])

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


(vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer)0x7f9118ee3670

In [9]:
col_df = client.materialize.query_table('allen_v1_column_types_slanted_ref')
col_df.head()

,id,created,valid,target_id,classification_system,cell_type,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position
0,50,2023-03-18 14:13:21.613360+00:00,t,258319,aibs_coarse_excitatory,23P,258319,2020-09-28 22:40:42.476911+00:00,t,261.806162,89309001002848425,864691135698459925,"[178400, 143248, 21238]","[nan, nan, nan]","[nan, nan, nan]"
1,1119,2023-03-18 14:13:22.506660+00:00,t,276438,aibs_coarse_excitatory,6P-CT,276438,2020-09-28 22:40:42.700226+00:00,t,277.317714,89465269428261699,864691136487559186,"[179648, 258768, 23597]","[nan, nan, nan]","[nan, nan, nan]"
2,35,2023-03-18 14:13:21.602813+00:00,t,260552,aibs_coarse_excitatory,23P,260552,2020-09-28 22:40:42.745779+00:00,t,230.111805,89170256379033022,864691136040432126,"[177408, 157968, 21002]","[nan, nan, nan]","[nan, nan, nan]"
3,95,2023-03-18 14:13:21.644304+00:00,t,260263,aibs_coarse_excitatory,23P,260263,2020-09-28 22:40:42.746658+00:00,t,274.324193,88044356338331571,864691136334881203,"[169440, 158128, 20266]","[nan, nan, nan]","[nan, nan, nan]"
4,81,2023-03-18 14:13:21.634505+00:00,t,262898,aibs_coarse_inhibitory,BPC,262898,2020-09-28 22:40:42.749245+00:00,t,230.092308,88468836747612860,864691135654475970,"[172512, 175280, 21964]","[nan, nan, nan]","[nan, nan, nan]"


In [14]:
import numpy as np

In [15]:
root_pt = [178400, 143248, 21238]
root_pt_nm = root_pt * np.array([4,4,40])

In [16]:
comp_mask = mesh_filters.filter_largest_component(mesh)
mask_filt = mesh.apply_mask(comp_mask)

In [18]:
comp_mask.shape, mesh.vertices.shape

((1573926,), (1573926, 3))

In [20]:
soma_mask = mesh_filters.filter_spatial_distance_from_points(
            mask_filt,
            root_pt_nm,
            15_000
)

soma_mesh = mask_filt.apply_mask(soma_mask)

In [22]:
neuron_actor = trimesh_vtk.mesh_actor(mesh,
                                     opacity = 0.5,
                                     color = (0,1,1))

soma_actor = trimesh_vtk.mesh_actor(soma_mesh,
                                   opacity = 1,
                                   color = (1,0,1))

trimesh_vtk.render_actors([neuron_actor, soma_actor])

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


(vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer)0x7f90b270f910

In [24]:
mesh.is_watertight

False

In [ ]:
mesh.fix_mesh().is_watertight

# Task 2

- Using your mesh from Task 1, use mask filtering to get a 150,000nm cutout around the soma
- Visualize this cutout and ensure it looks correct. What ultrastructural features are available to you just from the mesh? Is there anything you would like to measure?
- What is the area and volume of you cutout?
- Now that you have done this for one cell, do any of the metrics you've measure vary across cells?

#### Tip 1: Remember that meshes need to be watertight in order to trust the volume and area measurements


#### Bonus Task: 
- What about synapses? Can you query your cell of interest and measure the density of synapses within 15 microns of the cell's soma center?